In [1]:
# Check if Nvdia GPu
!nvidia-smi

Thu Dec  4 19:10:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Preparation
1. Upload zipped data file.
2. Unzip data
3. Split data into training and validation sets.


In [3]:
!unzip -q /content/pole-dataset.zip -d /content/pole-dataset

# Also install dependencies
!pip3 install ultralytics --quiet

In [4]:
# Sort dataset into training and validation sets
import random
import os
import shutil


TRAIN_PERCENTAGE = 0.9
DATA_BASE_DIR   = "/content/pole-dataset/"
IMG_DIR         = "images/"
LABEL_DIR       = "labels/"
TRAIN_BASE_DIR  = "/content/pole-dataset-sorted/"
DEBUG           = True

def dbg_print(msg):
    if DEBUG: print(f"[DEBUG]: {msg}")

def shuffle_files(filename_list):
    random.shuffle(filename_list)
    return filename_list

def copy_files(file_list, split, img_src_dir, lbl_src_dir, base_out_dir):
    img_out_dir = os.path.join(base_out_dir, split, "images")
    lbl_out_dir = os.path.join(base_out_dir, split, "labels")
    os.makedirs(img_out_dir, exist_ok=True)
    os.makedirs(lbl_out_dir, exist_ok=True)

    for fname in file_list:
        shutil.copy(os.path.join(img_src_dir, fname),
                    os.path.join(img_out_dir, fname))
        filename, _ = os.path.splitext(fname)
        lbl_name = f"{filename}.txt"
        lbl_path = os.path.join(lbl_src_dir, lbl_name)
        if os.path.exists(lbl_path):
            shutil.copy(lbl_path, os.path.join(lbl_out_dir, lbl_name))
        else:
            dbg_print(f"No label found for {fname}, skipping.")

def main():
    img_src_dir = os.path.join(DATA_BASE_DIR, IMG_DIR)
    lbl_src_dir = os.path.join(DATA_BASE_DIR, LABEL_DIR)
    images = [f for f in os.listdir(img_src_dir)]
    images = shuffle_files(images)
    train_size = int(len(images) * TRAIN_PERCENTAGE)
    train_files = images[:train_size]
    val_files = images[train_size:]
    copy_files(train_files, "train", img_src_dir, lbl_src_dir, TRAIN_BASE_DIR)
    copy_files(val_files, "validation", img_src_dir, lbl_src_dir, TRAIN_BASE_DIR)
    print(f"Done. Train: {len(train_files)} images, Validation: {len(val_files)} images.")

if __name__ == "__main__":
    main()

Done. Train: 111 images, Validation: 13 images.


### Training
We can now train our YOLOv5 model after sorting our dataset.

But first, we need to create our Ultralytics config file.

In [5]:
# Automatically create data.yaml config file
# 1. Reads "classes.txt" file to get list of class names
# 2. Creates data dictionary with correct paths to folders, number of classes, and names of classes
# 3. Writes data in YAML format to data.yaml

import yaml
import os


CLASSES_TXT      = "/content/pole-dataset/classes.txt"
DATA_YAML_PATH   = "/content/pole-dataset-sorted/data.yaml"
DATA_BASE_DIR    = "/content/pole-dataset-sorted/"
DEBUG            = True

def dbg_print(msg):
    if DEBUG:
        print(f"[DEBUG]: {msg}")

def read_classes(path_to_classes_txt):
    if not os.path.exists(path_to_classes_txt):
        print(f"classes.txt file not found! Please create one at {path_to_classes_txt}")
        return []
    classes = []
    with open(path_to_classes_txt, "r") as f:
        for line in f.readlines():
            if len(line.strip()) == 0:
                continue
            classes.append(line.strip())
    dbg_print(f"Loaded {len(classes)} classes: {classes}")
    return classes

def create_data_yaml(classes, path_to_data_yaml):
    data = {
        "path": DATA_BASE_DIR,
        "train": "train/images",
        "val": "validation/images",
        "nc": len(classes),
        "names": classes
    }
    with open(path_to_data_yaml, "w") as f:
        yaml.dump(data, f, sort_keys=False)
    print(f"Created config file at {path_to_data_yaml}")

def main():
    classes = read_classes(CLASSES_TXT)
    if not classes:
        return
    create_data_yaml(classes, DATA_YAML_PATH)

if __name__ == "__main__":
    main()


[DEBUG]: Loaded 1 classes: ['Pole']
Created config file at /content/pole-dataset-sorted/data.yaml


In [9]:
# Train YOLO model
!yolo detect train data=/content/pole-dataset-sorted/data.yaml model=yolov5su.pt epochs=75 imgsz=640

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/pole-dataset-sorted/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=75, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov5su.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=

### Validate Model Results
Check `runs/detect/predict` for results.

In [11]:
# Change training iteration as needed
!yolo detect predict model=runs/detect/train2/weights/best.pt source=/content/pole-dataset-sorted/validation/images save=True

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv5s summary (fused): 84 layers, 9,111,923 parameters, 0 gradients, 23.8 GFLOPs

image 1/13 /content/pole-dataset-sorted/validation/images/082b07b8-13.png: 640x480 1 Pole, 47.6ms
image 2/13 /content/pole-dataset-sorted/validation/images/0d39245b-78.png: 640x480 3 Poles, 11.2ms
image 3/13 /content/pole-dataset-sorted/validation/images/15ab5253-73.png: 640x480 1 Pole, 11.5ms
image 4/13 /content/pole-dataset-sorted/validation/images/22a22f8f-92.png: 640x480 1 Pole, 11.2ms
image 5/13 /content/pole-dataset-sorted/validation/images/29f4e7f9-43.png: 640x480 1 Pole, 11.2ms
image 6/13 /content/pole-dataset-sorted/validation/images/4c9f3eef-108.png: 640x480 2 Poles, 11.2ms
image 7/13 /content/pole-dataset-sorted/validation/images/4ea9970b-120.png: 640x480 3 Poles, 11.2ms
image 8/13 /content/pole-dataset-sorted/validation/images/519de9a7-123.png: 640x480 3 Poles, 11.2ms
image 9/13 /content/pole-dataset-sorted/va